In [56]:
#

import sqlite3
import os
import pandas as pd
from datetime import datetime, time, timedelta
from pytz import timezone, UTC
from airportsdata import load
import matplotlib.pyplot as plt
import pytz

# Global variables

mainbase = ['SGN', 'HAN', 'DAD', 'CXR', 'HPH','VII','PQC','VCA']
aircraft_types = ['A320', 'A321', 'A330']
airports = ['SGN', 'HAN', 'DAD', 'CXR']
file_path = '/Users/dongthan/Desktop/moc/acreadiness/csv/airports.csv'

## Bảng thông số tông số máy bay của hãng

vjc_fleet_total_ac = 86
a320_fleet_total= 18
a321_fleet_total  = 61
a330_fleet_total = 7
a320_wet_lease_total = 4
##
a321_ceo_fleet_total = 36
a321_neo_fleet_total = 11
a321_neo_acf_fleet_total = 14
##

total_ac_available = vjc_fleet_total_ac + a320_wet_lease_total

In [57]:
# Ép kiểu dữ liệu về định dạng HH:MM
def normalize_time_format(value):
    if pd.isnull(value):
        return None
    elif isinstance(value, (int, float)):
        # Chuyển đổi số thành chuỗi định dạng giờ
        return f"{int(value):02d}:00:00"
    elif isinstance(value, str):
        # Xử lý các định dạng chuỗi khác nhau
        try:
            datetime_obj = pd.to_datetime(value, format='%H:%M:%S.%f')
        except ValueError:
            try:
                datetime_obj = pd.to_datetime(value, format='%H:%M:%S')
            except ValueError:
                try:
                    datetime_obj = pd.to_datetime(value, format='%H:%M')
                except ValueError:
                    try:
                        datetime_obj = pd.to_datetime(value, format='%I:%M:%S %p')
                    except ValueError:
                        print(f"Could not convert time: {value}")
                        return None
        return datetime_obj.strftime('%H:%M:%S')
    elif isinstance(value, (datetime, time)):
        # Chuyển đổi datetime hoặc time thành chuỗi định dạng giờ
        return value.strftime('%H:%M:%S')
    elif isinstance(value, timedelta):
        # Chuyển đổi timedelta thành chuỗi định dạng giờ
        hours = value.total_seconds() // 3600
        minutes = (value.total_seconds() % 3600) // 60
        seconds = value.total_seconds() % 60
        return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"
    else:
        print(f"Unsupported data type: {type(value)}")
        return None
## Hàm chuyển đổi nhiều cột thời gian về định dạng chuẩn HH:MM

def normalize_time_columns(df, columns):
    for col in columns:
        df[col] = df[col].apply(normalize_time_format)
    return df

## Hàm chuyển đổi thời gian về đơn vị phút HH:MM -> MÍNUTES
def convert_to_minutes(value):
    if pd.isnull(value):
        return None
    elif isinstance(value, (int, float)):
        return int(value)
    elif isinstance(value, str):
        try:
            time_obj = pd.to_datetime(value, format='%H:%M:%S.%f').time()
        except ValueError:
            try:
                time_obj = pd.to_datetime(value, format='%H:%M:%S').time()
            except ValueError:
                try:
                    time_obj = pd.to_datetime(value, format='%H:%M').time()
                except ValueError:
                    print(f"Could not convert time: {value}")
                    return None
        return time_obj.hour * 60 + time_obj.minute
    elif isinstance(value, (datetime, time)):
        return value.hour * 60 + value.minute
    elif isinstance(value, timedelta):
        return value.total_seconds() // 60
    else:
        print(f"Unsupported data type: {type(value)}")
        return None
## Hàm chuyển đổi nhiều cột thời gian về đơn vị phút
def convert_many_cols_to_minutes(df, columns):
    for column in columns:
        df[column] = df[column].apply(convert_to_minutes)
    return df

def convert_to_time_string(value):
    if pd.isnull(value):
        return None
    elif isinstance(value, (int, float)):
        minutes = int(value)
        return f"{minutes // 60:02d}:{minutes % 60:02d}"
    elif isinstance(value, str):
        return value
    elif isinstance(value, (datetime, time)):
        return value.strftime('%H:%M')
    elif isinstance(value, timedelta):
        minutes = value.total_seconds() // 60
        return f"{int(minutes // 60):02d}:{int(minutes % 60):02d}"
    else:
        print(f"Unsupported data type: {type(value)}")
        return None

def convert_many_cols_to_time_string(df, columns):
    for col in columns:
        df[col] = df[col].apply(convert_to_time_string)
    return df

In [58]:
def check_df(df):
    """
    Display the first 10 and last 10 rows, information, summary statistics, shape, and columns of a DataFrame.
    
    Args:
    df (pandas.DataFrame): Input DataFrame
    """
    display(df.shape)
    display(df.columns)
    display(df.head(10))
    display(df.tail(10))
    display(df.info())
    display(df.describe())

In [59]:
df = pd.read_excel('/Users/dongthan/Desktop/moc/acreadiness/charter_china_2024.xlsx') # Load data from excel file "Charter Trung quốc"

In [60]:

df1 = df.copy()

# rename columns

df1.rename(columns={'A/C':'AC','OPERATION  PERIOD（FINAL)':'OPS_PRERIOD_FINAL','FLIGHT \n NO':'FLT_NO','OPERATING DATE':'OPS_DATE','FINAL\n LOCAL TIME':'STD','Unnamed: 8':'STA'}, inplace=True)
# Lấy các cols cần thiết

cols = ['AC','OPS_PRERIOD_FINAL','FLT_NO','SECTOR','STD','STA','OPS_DATE']

df1= df1[cols]
# Remove first row

df1 = df1.iloc[1:]

# Xử lý cột OPS_PRERIOD_FINAL = Tách thành 2 col là OPS và OPS_PERIOD_FINAL_FROM và OPS_PERIOD_FINAL_TO. Split the column OPS_PRERIOD_FINAL into 2 columns: OPS and PERIOD_FINAL_FROM and PERIOD_FINAL_TO. by "-"
# Split 'OPS_PRERIOD_FINAL' into 'FINAL_FR' and 'FINAL_TO'
df1[['FINAL_FR', 'FINAL_TO']] = df1['OPS_PRERIOD_FINAL'].str.split('-', n=1, expand=True)

# Drop the original 'OPS_PRERIOD_FINAL' column
df1 = df1.drop(columns=['OPS_PRERIOD_FINAL'])

# Convert 'FINAL_FR' and 'FINAL_TO' to datetime, errors='coerce' will convert invalid values to NaT
df1['FINAL_FR'] = pd.to_datetime(df1['FINAL_FR'], format='%d %b %Y', errors='coerce')
df1['FINAL_TO'] = pd.to_datetime(df1['FINAL_TO'], format='%d %b %Y', errors='coerce')




In [61]:
# Tách cột SECTOR thành 2 cột DEP và ARR. Split 'SECTOR' into 'DEP' and 'ARR' by "-"
df1[['DEP', 'ARR']] = df1['SECTOR'].str.split('-', n=1, expand=True)

In [62]:
df1

,AC,FLT_NO,SECTOR,STD,STA,OPS_DATE,FINAL_FR,FINAL_TO,DEP,ARR
1,A321 CEO,VJ5348,CXR-HGH,15:45:00,20:20:00,"1.3.5.7(Mon,Wed,Fri,Sun)",2024-03-31,2024-10-26,CXR,HGH
2,A321 CEO,VJ5349,HGH-CXR,21:35:00,1900-01-01 00:20:00,NaN,NaT,NaT,HGH,CXR
3,A321 CEO,VJ5346,CXR-HGH,15:45:00,20:20:00,".2.4.6.(Tue,Thu,Sat)",2024-04-09,2024-10-26,CXR,HGH
4,A321 CEO,VJ5347,HGH-CXR,21:35:00,1900-01-01 00:20:00,NaN,NaT,NaT,HGH,CXR
5,A321 CEO,VJ3346,PQC-HGH,16:00:00,21:00:00,4/9/14/18/23/28JUL；1/6/11/15/20/25/29AUG；3/8/1...,NaT,2024-10-26,PQC,HGH
...,...,...,...,...,...,...,...,...,...,...
150,A321 CEO,VJ8893,MFM-DAD,18:00:00,18:40:00,NaN,NaT,NaT,MFM,DAD
151,A321 CEO,VJ5530,CXR-MFM,10:55:00,14:00:00,Daily,2024-03-31,2024-10-26,CXR,MFM
152,A321 CEO,VJ7581,MFM-HAN,15:00:00,15:45:00,NaN,NaT,NaT,MFM,HAN
153,A321 CEO,VJ7580,HAN-MFM,17:55:00,20:40:00,Daily,2024-03-31,2024-10-26,HAN,MFM


In [63]:
# Normalize the time columns 'STD' and 'STA' to HH:MM format
df1 = normalize_time_columns(df1, ['STD', 'STA'])

In [64]:
# reoder columns

df1 = df1[['AC','FLT_NO','SECTOR','DEP','ARR','STD','STA','FINAL_FR','FINAL_TO','OPS_DATE']]

In [65]:
# Tạo DataFrame cho các hàng lẻ
df_odd = df1.iloc[::2].reset_index(drop=True)
df_odd.columns = [col + '_DEP' if col not in ['AC', 'FLT_NO'] else col for col in df_odd.columns]

# Tạo DataFrame cho các hàng chẵn
df_even = df1.iloc[1::2].reset_index(drop=True)
df_even.columns = [col + '_RT' if col not in ['AC'] else col for col in df_even.columns]

# Ghép hai DataFrame lại với nhau
df_combined = pd.concat([df_odd, df_even], axis=1)

In [75]:
df_combined.columns

Index(['AC', 'FLT_NO', 'SECTOR_DEP', 'DEP_DEP', 'ARR_DEP', 'STD_DEP',
       'STA_DEP', 'FINAL_FR_DEP', 'FINAL_TO_DEP', 'OPS_DATE_DEP', 'AC',
       'FLT_NO_RT', 'SECTOR_RT', 'DEP_RT', 'ARR_RT', 'STD_RT', 'STA_RT',
       'FINAL_FR_RT', 'FINAL_TO_RT', 'OPS_DATE_RT'],
      dtype='object')

In [67]:
df_combined.shape[0]

77

In [69]:
df_combined.to_excel('/Users/dongthan/Desktop/moc/acreadiness/charter_china_2024_cleaned.xlsx', index=False)

In [80]:
import pandas as pd
import re

# Tạo DataFrame mới với cột 'DATE'
charter = pd.DataFrame(pd.date_range(start='2024-04-01', end='2024-12-31'), columns=['DATE'])

# Duyệt qua từng hàng trong df_combined
for idx, row in df_combined.iterrows():
    ops_date_dep = row['OPS_DATE_DEP']
    
    # Xử lý từng trường hợp
    if 'Daily' in ops_date_dep:
        charter.loc[:, row['FLT_NO']] = True
    elif re.match(r"\d(\.\d)*\(", ops_date_dep):
        days_of_week = [int(i) for i in ops_date_dep if i.isdigit()]
        charter.loc[:, row['FLT_NO']] = charter['DATE'].dt.dayofweek.isin(days_of_week)
    elif re.search(r"\d{1,2}[A-Z]{3}", ops_date_dep):
        dates = re.findall(r"\d{1,2}[A-Z]{3}", ops_date_dep)
        date_list = [pd.to_datetime(date + '2024', format='%d%b%Y') for date in dates]
        charter.loc[charter['DATE'].isin(date_list), row['FLT_NO']] = True
    elif 'Addition' in ops_date_dep:
        dates = re.findall(r"\d{1,2}[A-Z][a-z]{2}", ops_date_dep)
        date_list = [pd.to_datetime(date + '2024', format='%d%b%Y') for date in dates]
        charter.loc[charter['DATE'].isin(date_list), row['FLT_NO']] = True
    elif 'Cancel' in ops_date_dep:
        dates = re.findall(r"\d{1,2}[A-Z][a-z]{2}", ops_date_dep)
        date_list = [pd.to_datetime(date + '2024', format='%d%b%Y') for date in dates]
        charter.loc[charter['DATE'].isin(date_list), row['FLT_NO']] = False
    else:
        charter.loc[:, row['FLT_NO']] = pd.NA

# Chuyển đổi DataFrame thành định dạng mong muốn
charter = charter.melt(id_vars='DATE', var_name='FLT_NO', value_name='FLIGHT_EXISTS')
charter = charter.merge(df_combined, on='FLT_NO')
charter = charter[charter['FLIGHT_EXISTS'] == True]
charter.drop('FLIGHT_EXISTS', axis=1, inplace=True)
charter['DATE'] = charter['DATE'].dt.strftime('%d.%b.%Y')

In [82]:
charter.to_excel('/Users/dongthan/Desktop/moc/acreadiness/charter_china_2024_cleaned.xlsx', index=False)
